In [1]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

# Milliman PAC File ETL
1) log into SFTP site: https://secureftp.milliman.com  
2) In the "from milliman" folder, go to the latest directory  
     - In this case, the latest directory is "20221216"  
3) Make sure you have access to the following folder: \\Main Street Health\MSH_Strategy_Group - ACO Analyses\Raw MIlliman Flat FIles\  
     - create folder with same datestamp as file from milliman, in this case "20221216"  
4) Download the files from the SFTP site  
     - in this case there are 3  
5) copy the data disctionary to the folder on the network drive.  
6) download the prospective and retrospective CSV files into database tables in both the MSH and CB datagrip environments (Alan has a place he puts them)  
7) delete the last 30 columns from these database tables (we will create a "pivot file" with these 30 columns):  
8) with these 30 columns, run Alan's "pivot code" to create these files:  
     - strategic_milliman_prosp_20221128_pivot
     -  strategic_milliman_retro_20221128_pivot
9) copy 4 files to the "20221216" folder  
     - prospective flat file (without the 30 MA columns)
     - retrospective flat file (without the 30 MA columns)
     - prospective MA pivot file
     - retrospective MA pivor file
10) email Ali and Ben that process is complete with the following summaries that you can pull via queries and paste in excel into the same folder.  

### Config

In [2]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20231212'
pro_file_name = 'ACO Builder MSSP Prospective Explorer No MD Visit Req v2024.3.txt'
retro_file_name = 'ACO Builder MSSP Retrospective Explorer No MD Visit Req v2024.3.txt'

### Pro load

In [3]:
pro_df = pd.read_csv(f'{data_dir}/{current_folder}/{pro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
pro_df = cb_utils.df_format_columns(pro_df)
print(f'{pro_df.shape[0]} rows, {pro_df.shape[1]} columns')
pro_df.head()

711630 rows, 32 columns


,pac_id,deid_tin,prvdr_name,year,program_type,mms_esrd,mms_dis,mms_ad,mms_and,mms_total,rs_esrd,rs_dis,rs_ad,rs_and,risk_composite,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro
0,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2017,MSSP Prospective,24.0,127.0,91.0,786.0,1028,1.246027,1.053987,0.743116,1.192508,1.149579,4486.701037,1541.033458,1227.484241,1592.224074,1621.187999,6422.618344,971.738226,1453.909645,855.287752,1034.153660,1.363671,6190.214378,926.960749,1414.578839,829.504110,1000.500081,1.409540
1,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2018,MSSP Prospective,12.0,164.0,169.0,1275.0,1620,0.775247,1.726701,1.030603,1.093249,1.129966,4806.024031,1316.571793,2488.021439,1422.827226,1548.253332,6401.790718,980.975629,1485.288591,896.260666,1008.078954,1.359196,6588.785764,895.084955,1418.841402,857.921752,960.226328,1.426931
2,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2019,MSSP Prospective,0.0,229.0,128.0,1134.0,1491,NaN,1.321597,1.503569,1.176729,1.238518,NaN,1086.645206,1408.122757,976.842764,1030.731831,NaN,1033.170868,1453.326795,947.102605,1005.188258,0.827935,NaN,951.267784,1395.566254,905.352361,955.575731,0.870920
3,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2020,MSSP Prospective,12.0,208.0,131.0,1311.0,1662,0.702839,1.421957,0.809204,1.033532,1.079534,3627.703418,1436.731663,816.391181,945.456704,1016.133340,6493.866590,985.203048,1577.222921,891.456731,959.344803,0.981159,6687.820022,878.592349,1428.220968,850.629683,903.046192,1.042328
4,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2021,MSSP Prospective,NaN,220.0,110.0,1218.0,1548,0.935558,1.365572,1.009259,1.006248,1.072736,2840.723750,1281.514409,990.256818,934.094249,987.460087,6654.828765,971.299281,1378.755186,929.492779,953.914452,0.964978,6338.796906,926.928509,1360.255980,893.385741,918.001423,1.002728


In [4]:
conn = cb_utils.get_engine(source='msh_analytics')
pro_df.to_sql(f'milliman_pro_no_md_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # , if_exists='replace'

711630

In [5]:
ret_df = pd.read_csv(f'{data_dir}/{current_folder}/{retro_file_name}',  encoding='latin1', low_memory=False, delimiter='\t')
ret_df = cb_utils.df_format_columns(ret_df)
print(f'{ret_df.shape[0]} rows, {ret_df.shape[1]} columns')
ret_df.head()

691123 rows, 32 columns


,pac_id,deid_tin,prvdr_name,year,program_type,mms_esrd,mms_dis,mms_ad,mms_and,mms_total,rs_esrd,rs_dis,rs_ad,rs_and,risk_composite,paid_esrd,paid_dis,paid_ad,paid_and,paid_composite,reg_ret_exp_esrd,reg_ret_exp_dis,reg_ret_exp_ad,reg_ret_exp_and,reg_ret_exp_composite,regional_efficiency_ret,reg_pro_exp_esrd,reg_pro_exp_dis,reg_pro_exp_ad,reg_pro_exp_and,reg_pro_exp_composite,regional_efficiency_pro
0,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2016,MSSP Retrospective,24.0,206.0,173.0,1313.0,1716,1.090726,0.971947,0.849591,1.073577,1.039898,4715.462628,1458.454427,1856.114031,1713.535182,1739.272807,5827.437531,905.793140,1387.197402,855.340674,977.037120,1.711850,NaN,NaN,NaN,NaN,NaN,NaN
1,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2017,MSSP Retrospective,12.0,250.0,174.0,1805.0,2241,0.859486,1.433233,0.946422,1.229279,1.224837,2934.181769,1462.664500,1587.616993,1692.850230,1665.647625,6245.434358,993.532432,1462.243362,852.690082,930.172550,1.461979,NaN,NaN,NaN,NaN,NaN,NaN
2,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2018,MSSP Retrospective,12.0,323.0,185.0,1554.0,2074,0.775247,1.217963,1.497643,1.153835,1.200671,4806.024031,1463.421215,2310.076821,1518.970887,1599.904682,6401.790718,974.462100,1464.472430,901.920668,990.229857,1.345656,NaN,NaN,NaN,NaN,NaN,NaN
3,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2019,MSSP Retrospective,0.0,342.0,160.0,1508.0,2010,NaN,1.291056,1.117051,0.991496,1.048423,NaN,1054.832327,1238.552681,1178.032843,1161.887867,NaN,1046.043523,1483.259105,946.421710,1016.469788,1.090267,NaN,NaN,NaN,NaN,NaN,NaN
4,9.335446e+09,00004B7C4023,RINDFLEISCH FAMILY PRACTICE PLLC,2020,MSSP Retrospective,12.0,257.0,120.0,1327.0,1716,0.702839,1.133058,0.665621,0.944445,0.973295,3627.703418,1393.406361,318.074898,880.118517,936.902177,6493.866590,991.092451,1560.750987,893.150238,950.440769,1.012802,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ret_df.to_sql(f'milliman_ret_no_md_{current_folder}', conn, schema='raw', index=False, method='multi', chunksize=1000) # if_exists='replace', 

691123

In [9]:
ret_df.year.value_counts()

2016    93392
2017    90403
2018    87384
2019    84748
2020    82510
2021    79560
2022    76027
Name: year, dtype: int64

In [10]:
for yr in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    ret_df.loc[ret_df.year == yr].to_csv(f'{data_dir}/{current_folder}/ret_{yr}.csv',index=False)

# Did not use below

In [6]:
conn = cb_utils.get_engine()
pro_df.to_sql(f'milliman_pro_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # , if_exists='replace'

613556

### Retro load

In [33]:

ret_df.to_csv('ret.csv',index=False)

In [10]:
conn = cb_utils.get_engine()
ret_df.to_sql(f'milliman_ret_{current_folder}', conn, schema='strategic', index=False, method='multi', chunksize=1000) # if_exists='replace', 

594838

### Pivots

In [11]:
def pivot_ma_plan_enrollment(df):
    dfs = []
    for i in range(1, 16):
        name, enrollment = f'ma_plan_{i}_name', f'ma_plan_{i}_enrollment'
        plan = df[['deid_tin', 'pac_id', 'year', name, enrollment]]
        plan = plan.loc[~(plan[name].isna()) & ~(plan[enrollment].isna())]
        plan.columns = ['deid_tin', 'pac_id', 'year', 'ma_plan_name', 'ma_enrollment']
        dfs.append(plan)
    return pd.concat(dfs)

In [12]:
ret_pivot_df = pivot_ma_plan_enrollment(ret_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
ret_pivot_df.head()

In [13]:
pro_pivot_df = pivot_ma_plan_enrollment(pro_df)

KeyError: "['ma_plan_1_name', 'ma_plan_1_enrollment'] not in index"

In [ ]:
pro_pivot_df.head()

### Summaries 
### Prospective

In [ ]:
s1 = ret_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s1.head()

### Retro

In [ ]:
s2 = pro_df[['year', 'unique_members']].groupby('year', as_index=False).sum()
s2.head()

### Prospective

In [ ]:
s3 = pro_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s3.head()

### Retro

In [ ]:
s4 = ret_pivot_df[['year', 'ma_plan_name', 'ma_enrollment']].groupby(['year', 'ma_plan_name'], as_index=False).sum().sort_values('ma_enrollment', ascending=False)
s4.head()

### Outputs

In [ ]:
ret_df.loc[:, ret_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_ret_{current_folder}.csv', index=False)

In [ ]:
pro_df.loc[:, pro_df.columns[:-30]].to_csv(f'{data_dir}/{current_folder}/ours_milliman_pro_{current_folder}.csv', index=False)

In [ ]:
pro_pivot_df.to_csv(f'{data_dir}/{current_folder}/pro_pivot.csv', index=False)
ret_pivot_df.to_csv(f'{data_dir}/{current_folder}/ret_pivot.csv', index=False)

In [ ]:
with pd.ExcelWriter(f'{data_dir}/{current_folder}/summary.xlsx') as writer:
    s1.to_excel(writer, sheet_name='retrospective', index=False)
    s2.to_excel(writer, sheet_name='prospective', index=False)
    s3.to_excel(writer, sheet_name='prospective_pivot', index=False)
    s4.to_excel(writer, sheet_name='retrospective_pivot', index=False)